In [1]:
# Literal prompt template

In [2]:
from langchain_core.prompts import PromptTemplate
prompt = PromptTemplate.from_template('{topic}에 대해 금융 관련 짧은 조언을 해주세요.')
prompt.invoke({'topic':'투자'})

StringPromptValue(text='투자에 대해 금융 관련 짧은 조언을 해주세요.')

In [3]:
# Chat prompt template

In [6]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ('system', '당신은 유능한 금융 조언가입니다.'),
    ('user', '{topic}에 대해 금융 관련 조언을 해주세요.')
])
prompt.invoke({'topic':'주식'})

ChatPromptValue(messages=[SystemMessage(content='당신은 유능한 금융 조언가입니다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='주식에 대해 금융 관련 조언을 해주세요.', additional_kwargs={}, response_metadata={})])

In [7]:
# message Holder

In [8]:
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import HumanMessage
prompt = ChatPromptTemplate.from_messages([
    ('system', '당신은 유능한 금융 조언가입니다.'),
    MessagesPlaceholder('msgs')
])
prompt.invoke({'msgs':[HumanMessage(content='안녕하세요!')]})

ChatPromptValue(messages=[SystemMessage(content='당신은 유능한 금융 조언가입니다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='안녕하세요!', additional_kwargs={}, response_metadata={})])

In [9]:
prompt = ChatPromptTemplate.from_messages([
    ('system', '당신은 유능한 금융 조언가입니다.'),
    ('placeholder', '{msgs}')
])
prompt.invoke({'msgs':[HumanMessage(content='안녕하세요!')]})

ChatPromptValue(messages=[SystemMessage(content='당신은 유능한 금융 조언가입니다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='안녕하세요!', additional_kwargs={}, response_metadata={})])

In [10]:
# Fewshot prompt

In [12]:
example_prompt = PromptTemplate.from_template('질문 : {question}\n답번 : {answer}')
examples = [
    {
        'question':'주식 투자와 예금 중 어느 것이 더 수익률이 높은가?',
        'answer':'''
        후속 질문이 필요한가요: 네.
        후속 질문: 주식 투자의 평균 수익률은 얼마인가요?
        중간 답번: 주식 투자의 평균 수익률은 연 7%입니다.
        후속 질문: 예금의 평균 이자율은 얼마인가요?
        중간 답번: 예금의 평균 이자율은 연 1%입니다.
        따라서 최종 답변은: 주식 투자
        ''',
    },
    {
        'question':'부동산과 채권 중 어느 것이 더 안정적인 투자처인가?',
        'answer':'''
        후속 질문이 필요한가요: 네.
        후속 질문: 부동산 투자의 위험도는 어느 정도인가요?
        중간 질문: 부동산 투자의 위험도는 중간 수준입니다.
        후속 질문: 채권의 위험도는 어느 정도인가요?
        중간 질문: 채권의 위험도는 낮은 편입니다.
        따라서 최종 답변은: 채권
        '''
    },
]

In [13]:
print(example_prompt.invoke(examples[0]).to_string())

질문 : 주식 투자와 예금 중 어느 것이 더 수익률이 높은가?
답번 : 
        후속 질문이 필요한가요: 네.
        후속 질문: 주식 투자의 평균 수익률은 얼마인가요?
        중간 답번: 주식 투자의 평균 수익률은 연 7%입니다.
        후속 질문: 예금의 평균 이자율은 얼마인가요?
        중간 답번: 예금의 평균 이자율은 연 1%입니다.
        따라서 최종 답변은: 주식 투자
        


In [15]:
from langchain_core.prompts import FewShotPromptTemplate
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix='질문: {input}',
    input_variables=['input'],
)
print(prompt.invoke({'input':'부동산 투자의 장점은 무엇인가?'}).to_string())

질문 : 주식 투자와 예금 중 어느 것이 더 수익률이 높은가?
답번 : 
        후속 질문이 필요한가요: 네.
        후속 질문: 주식 투자의 평균 수익률은 얼마인가요?
        중간 답번: 주식 투자의 평균 수익률은 연 7%입니다.
        후속 질문: 예금의 평균 이자율은 얼마인가요?
        중간 답번: 예금의 평균 이자율은 연 1%입니다.
        따라서 최종 답변은: 주식 투자
        

질문 : 부동산과 채권 중 어느 것이 더 안정적인 투자처인가?
답번 : 
        후속 질문이 필요한가요: 네.
        후속 질문: 부동산 투자의 위험도는 어느 정도인가요?
        중간 질문: 부동산 투자의 위험도는 중간 수준입니다.
        후속 질문: 채권의 위험도는 어느 정도인가요?
        중간 질문: 채권의 위험도는 낮은 편입니다.
        따라서 최종 답변은: 채권
        

질문: 부동산 투자의 장점은 무엇인가?


In [21]:
from langchain_chroma import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv
import os

In [23]:
load_dotenv()
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    OpenAIEmbeddings(api_key=os.getenv('OPENAI_API_KEY')),
    Chroma,
    k=1,
)

In [24]:
question = '부동산 투자의 장점은 무엇인가?'
selected_examples = example_selector.select_examples({'question':question})

In [25]:
print(f'입력된 질문 ; {question}')
print(selected_examples)

입력된 질문 ; 부동산 투자의 장점은 무엇인가?
[{'question': '부동산과 채권 중 어느 것이 더 안정적인 투자처인가?', 'answer': '\n        후속 질문이 필요한가요: 네.\n        후속 질문: 부동산 투자의 위험도는 어느 정도인가요?\n        중간 질문: 부동산 투자의 위험도는 중간 수준입니다.\n        후속 질문: 채권의 위험도는 어느 정도인가요?\n        중간 질문: 채권의 위험도는 낮은 편입니다.\n        따라서 최종 답변은: 채권\n        '}]


In [30]:
for i in selected_examples:
    print('# 입력과 가장 유사한 예제는?')
    for k, v in i.items():
        print(f'{k}:{v}')

# 입력과 가장 유사한 예제는?
question:부동산과 채권 중 어느 것이 더 안정적인 투자처인가?
answer:
        후속 질문이 필요한가요: 네.
        후속 질문: 부동산 투자의 위험도는 어느 정도인가요?
        중간 질문: 부동산 투자의 위험도는 중간 수준입니다.
        후속 질문: 채권의 위험도는 어느 정도인가요?
        중간 질문: 채권의 위험도는 낮은 편입니다.
        따라서 최종 답변은: 채권
        


In [31]:
# Real useage

In [32]:
example_prompt = PromptTemplate(
    input_variables=['question', 'answer'],
    template='질문: {question}\n답변: {answer}')

In [34]:
prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="다음은 금융 관련 질문과 답변의 예입니다:",
    suffix="질문: {input}\n답변:",
    input_variables=['input']
)

In [35]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model='gpt-4o')
chain = prompt|model
response = chain.invoke({'input':'부동산 투자의 장점은 무엇인가?'})
print(response)

content='부동산 투자의 장점에 대해 설명드리겠습니다:\n\n1. **안정적인 수익**: 부동산은 임대 수익과 자산 가치 상승을 통해 안정적인 현금 흐름을 제공할 수 있습니다.\n\n2. **물가 상승 방어**: 부동산은 인플레이션에 대한 방어 역할을 합니다. 시간이 지남에 따라 자산 가치와 임대료가 물가 상승에 맞춰 상승할 가능성이 높습니다.\n\n3. **자산의 실물성**: 부동산은 실물 자산으로, 손에 잡히지 않는 금융 자산과는 달리 물리적으로 존재하는 자산입니다. 이는 투자자에게 심리적 안정을 줄 수 있습니다.\n\n4. **자산 다변화**: 포트폴리오에 부동산을 포함시키면 자산 다변화 효과를 통해 전반적인 투자 위험을 줄일 수 있습니다.\n\n5. **세금 혜택**: 각국 정부는 부동산 투자자에게 다양한 세금 이점을 제공하기도 합니다. 예를 들어, 이자 비용 또는 감가상각비를 공제할 수 있는 기회가 있을 수 있습니다.\n\n이러한 이점들은 부동산이 매력적인 투자 옵션으로 작용하도록 만듭니다. 다만, 부동산 투자에는 지역, 시장 상황, 관리 요구 사항 등 고려할 사항이 많으므로 신중한 접근이 필요합니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 312, 'prompt_tokens': 151, 'total_tokens': 463, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_ff25b2783a', 'i

In [36]:
# hub connect

In [37]:
from langchain import hub

In [38]:
prompt = hub.pull('hardkothari/prompt-maker')
prompt

/Users/janghongseo/PycharmProjects/RAG-Master/.venv/lib/python3.11/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


ChatPromptTemplate(input_variables=['lazy_prompt', 'task'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'hardkothari', 'lc_hub_repo': 'prompt-maker', 'lc_hub_commit_hash': 'c5db8eeefa7be4862a9599b759608dd10ee53f53910838f69abb5ab31c257c2d'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are an expert Prompt Writer for Large Language Models.\n\n'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['lazy_prompt', 'task'], input_types={}, partial_variables={}, template='Your goal is to improve the prompt given below for {task} :\n--------------------\n\nPrompt: {lazy_prompt}\n\n--------------------\n\nHere are several tips on writing great prompts:\n\n-------\n\nStart the prompt by stating that it is an expert in the subject.\n\nPut instructions at the beginning of the prompt and use ### or to separate the instruction and context \n\nBe specific, 